# Machine learning - LightGBM

In [2]:
import os
import pandas as pd

i=0
coins=[]
for file in [x for x in os.listdir("data/wrangled/") if "csv" in x]:
    print ("loading "+file)
    temp_df=pd.read_csv("data/wrangled/"+file)
    temp_df["coin"]=i 
    coins.append(file[:-13])
    if i:
        full_df=full_df.append(temp_df)
    else:
        full_df=temp_df
    i+=1

loading Stellar_wrangled.csv
loading Polkadot_wrangled.csv
loading Tron_wrangled.csv
loading Litecoin_wrangled.csv
loading WrappedBitcoin_wrangled.csv
loading ChainLink_wrangled.csv
loading Cardano_wrangled.csv
loading BinanceCoin_wrangled.csv
loading Tether_wrangled.csv
loading Aave_wrangled.csv
loading CryptocomCoin_wrangled.csv
loading Cosmos_wrangled.csv
loading Monero_wrangled.csv
loading NEM_wrangled.csv
loading XRP_wrangled.csv
loading Bitcoin_wrangled.csv
loading Ethereum_wrangled.csv
loading Solana_wrangled.csv
loading USDCoin_wrangled.csv
loading Dogecoin_wrangled.csv
loading Uniswap_wrangled.csv
loading Iota_wrangled.csv
loading EOS_wrangled.csv


In [25]:
full_df.sample()

,Performance,log_and_diffed_High,log_and_diffed_Low,log_and_diffed_Open,log_and_diffed_Close,log_and_diffed_Volume,log_and_diffed_Marketcap,coin
1620,1.771252,-0.014365,-0.010361,-0.029119,0.01575,-0.006315,0.015879,15


In [4]:
full_df.columns

Index(['Performance', 'log_and_diffed_High', 'log_and_diffed_Low',
       'log_and_diffed_Open', 'log_and_diffed_Close', 'log_and_diffed_Volume',
       'log_and_diffed_Marketcap', 'coin'],
      dtype='object')

In [5]:
len(full_df)

36079

In [29]:
full_df.reset_index(drop=True,inplace=True)

In [48]:
import numpy as np
full_df.replace([np.inf, -np.inf], np.nan, inplace=True)
full_df.dropna(inplace=True)
len(full_df)

36077

**1st attempt - 80-20 train/test split**

In [7]:
0.2-36079
#7215 results for testing

7215.8

In [49]:
import random

window=1

test_sample=random.sample(range(window,len(full_df)),k=7215)

y_test=full_df.iloc[test_sample,:]
X_test=[]

from tqdm import tqdm 

for i in tqdm(test_sample):
    for x in range(i-window,i):
        X_test.append(list(full_df.iloc[x,:]))


100%|██████████| 7215/7215 [00:01<00:00, 5945.94it/s]


In [50]:
X=full_df.drop(labels=test_sample,axis=0)
X_train=[]
Y_train=[]

from tqdm import tqdm 

for i in tqdm(range(window,len(full_df))):
    for x in range(i-window,i):
        X_train.append(list(full_df.iloc[x,:]))
    Y_train.append(list(full_df.iloc[i,:]))

100%|██████████| 36076/36076 [00:12<00:00, 2974.09it/s]


In [51]:
Y_train_df=pd.DataFrame(Y_train)

In [52]:
from lightgbm import LGBMRegressor

model= LGBMRegressor(boosting='dart',n_estimators=1000,max_depth=150,num_leaves=80,n_jobs=-1)

y_pred=[]
for i in tqdm(range(len(Y_train_df.columns))):
    model.fit(X=X_train, y=Y_train_df.iloc[:,i])
    y_pred.append(model.predict(X_test))

  0%|          | 0/8 [00:00<?, ?it/s]

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


 12%|█▎        | 1/8 [01:43<12:06, 103.75s/it]

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


 25%|██▌       | 2/8 [04:04<12:34, 125.81s/it]

...

 75%|███████▌  | 6/8 [12:28<04:05, 122.84s/it]

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


 88%|████████▊ | 7/8 [14:23<02:00, 120.08s/it]

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart


100%|██████████| 8/8 [15:56<00:00, 119.51s/it]


In [54]:
from sklearn.metrics import mean_squared_error

y_test_df=pd.DataFrame(y_test)

for i in range(len(y_pred)):
    print(mean_squared_error(y_test_df.iloc[:,i],y_pred[i],squared=False))

5.130845030574543
0.04100996914412226
0.035405384822692704
0.006016002868956268
0.047757749997126335
0.4000253337227069
0.05229695748076511
0.12663493305660345


In [68]:
y_test_df.iloc[0,:]

Performance                 -1.314999
log_and_diffed_High          0.080495
log_and_diffed_Low           0.215327
log_and_diffed_Open          0.224496
log_and_diffed_Close        -0.018203
log_and_diffed_Volume        0.102467
log_and_diffed_Marketcap    -0.018203
coin                        21.000000
Name: 33434, dtype: float64

In [69]:
y_pred_df=pd.DataFrame(y_pred)
y_pred_df.iloc[:,0]

0     5.227814
1     0.098988
2     0.199357
3     0.216423
4     0.035886
5     0.128101
6     0.053900
7    20.788705
Name: 0, dtype: float64

In [ ]:
model.save_model('model1.txt')
# model = lgb.Booster(model_file='lgb_classifier.txt')

Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, 
                         X, 
                         y, 
                         scoring='neg_root_mean_squared_error', 
                         cv=10, verbose=2)
np.mean(-scores)